In [1]:
import pandas as pd
import numpy as np

In [2]:
#loan= pd.read_csv("Final.csv",low_memory=False)

In [6]:

#loan['Int_Rate'] = loan['Int_Rate']*100

#loan = loan[np.isfinite(loan['Int_Rate'])]


#loan.head()
#loan.to_csv("Loan_Data.csv")

In [21]:
loan = pd.read_csv("Loan_Data.csv")

print(loan['Int_Rate'][0])
loan.head()


5.32


,Int_Rate,Investment,Term,Grade,Employment_Len,Home_Ownership,Annual_Income,Purpose,State,Debt_To_Income,Delinquance_2year
0,5.32,4800.0,36,A,2 years,MORTGAGE,143000.0,medical,KY,7.88,0
1,16.02,16000.0,60,C,9 years,MORTGAGE,40000.0,debt_consolidation,TN,32.01,0
2,11.99,32600.0,36,B,< 1 year,RENT,55000.0,house,CA,25.41,0
3,6.08,10000.0,36,A,7 years,MORTGAGE,54000.0,car,NY,0.56,0
4,5.32,9000.0,36,A,10+ years,OWN,158000.0,credit_card,NY,6.74,0


In [ ]:
loan['Int_Rate']=loan['Int_Rate'].round(decimals=2)

In [18]:
#loan.to_csv("Loan_Data.csv",index=False)

## DATA PREPROCESSING

In [3]:
from sklearn.preprocessing import LabelEncoder, StandardScaler


for column in loan.columns:
    
    if loan[column].dtype == type(object):
        le = LabelEncoder()
        loan[column]=le.fit_transform(loan[column].astype(str))
    print(loan[column].dtype)
        
        
    


float64
float64
int64
int64
int64
int64
float64
int64
int64
float64
int64


In [7]:
#loan.columns
#loan.count()


## GET RID OF THE ROWS WITH NANs IN THIS COLUMN
loan = loan[np.isfinite(loan['Debt_To_Income'])]

loan.count()


Int_Rate             464745
Investment           464745
Term                 464745
Grade                464745
Employment_Len       464745
Home_Ownership       464745
Annual_Income        464745
Purpose              464745
State                464745
Debt_To_Income       464745
Delinquance_2year    464745
dtype: int64

In [8]:


#x are entire dataset fields short of the one we are predicting(Interest rate), dropped 
X=loan.drop("Int_Rate",axis=1)
y=loan["Int_Rate"]
print(X.shape,y.shape)

(464745, 10) (464745,)


In [9]:
#import deeplearning modules
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical

#split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

X_scaler = StandardScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)

encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)


# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

Using TensorFlow backend.


In [13]:
#import statements
from keras.models import Sequential
from keras.layers import Dense


#add more layers where needed
model = Sequential()

model.add(Dense(units=100, activation='relu', input_dim=10))

model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))


#**********need to change number of nodes after I'm able to scale all X categories*****
model.add(Dense(units=67, activation='softmax'))

In [14]:
#Compile the model - CHANGE EPOCHS
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)


Epoch 1/60
 - 37s - loss: 1.9710 - acc: 0.2146
Epoch 2/60
 - 19s - loss: 1.9254 - acc: 0.2257
Epoch 3/60
 - 19s - loss: 1.9203 - acc: 0.2285
Epoch 4/60
 - 18s - loss: 1.9174 - acc: 0.2297
Epoch 5/60
 - 18s - loss: 1.9163 - acc: 0.2310
Epoch 6/60
 - 19s - loss: 1.9149 - acc: 0.2319
Epoch 7/60
 - 19s - loss: 1.9139 - acc: 0.2319
Epoch 8/60
 - 18s - loss: 1.9138 - acc: 0.2321
Epoch 9/60
 - 19s - loss: 1.9135 - acc: 0.2333
Epoch 10/60
 - 19s - loss: 1.9129 - acc: 0.2331
Epoch 11/60
 - 20s - loss: 1.9124 - acc: 0.2337
Epoch 12/60
 - 20s - loss: 1.9120 - acc: 0.2334
Epoch 13/60
 - 19s - loss: 1.9114 - acc: 0.2341
Epoch 14/60
 - 19s - loss: 1.9121 - acc: 0.2337
Epoch 15/60
 - 20s - loss: 1.9121 - acc: 0.2339
Epoch 16/60
 - 19s - loss: 1.9121 - acc: 0.2345
Epoch 17/60
 - 19s - loss: 1.9105 - acc: 0.2345
Epoch 18/60
 - 19s - loss: 1.9111 - acc: 0.2353
Epoch 19/60
 - 19s - loss: 1.9105 - acc: 0.2348
Epoch 20/60
 - 19s - loss: 1.9103 - acc: 0.2341
Epoch 21/60
 - 19s - loss: 1.9102 - acc: 0.2352
E

In [15]:
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)

print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 1.9192503390303206, Accuracy: 0.23054214326621889


In [16]:
#prediction testing
encoded_predictions = model.predict_classes(X_test_scaled[:5])

prediction_labels = label_encoder.inverse_transform(encoded_predictions)

print(f"Predicted Interest Rate: {prediction_labels}")
print(f"Actual Interest Rate: {list(y_test[:5])}")

Predicted Interest Rate: [ 15.05  12.62  15.05   7.35   5.32]
Actual Interest Rate: [15.050000000000001, 14.08, 13.59, 5.3199999999999985, 7.9699999999999989]


In [17]:
# Save the model
model.save("loan_model_trained.h5")

## Logistical Regression Testing


In [18]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [19]:
classifier.fit(X_train, y_train)

ValueError: Unknown label type: 'continuous'

In [ ]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

In [ ]:
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")